In [113]:
import requests
import json
import pandas as pd
from pandas import json_normalize

# Phenopacket format

In [114]:
data = pd.read_csv('case.csv')
data.head()

,id,patient_id,gene_id,features
0,patient_1,Patient:SCV000492960,Entrez:10000,"HP:0005306,HP:0000256,HP:0001263,HP:0002126"
1,patient_2,Patient:SCV000747622,Entrez:10002,"HP:0000505,HP:0000666,HP:0000551,HP:0000556"
2,patient_3,Patient:SCV000599197,Entrez:10002,HP:0000556
3,patient_4,Patient:SCV000598737,Entrez:1001,HP:0007754
4,patient_5,Patient:293197,Entrez:100151683,"HP:0001631,HP:0001601,HP:0011344,HP:0011994"


In [120]:
a = {'id': 'patient_1', 'subject': {'id': 'Patient:SCV000492960'}, 'phenotypicFeatures': [{'type': {'id': 'HP:0005306', 'label': ''}}, {'type': {'id':'HP:0000256', 'label': ''}}, {'type': {'id':'HP:0001263', 'label': '' }}, {'type':{'id':'HP:0002126', 'label': ''}}]}
a
#check documentation

{'id': 'patient_1',
 'subject': {'id': 'Patient:SCV000492960'},
 'phenotypicFeatures': [{'type': {'id': 'HP:0005306', 'label': ''}},
  {'type': {'id': 'HP:0000256', 'label': ''}},
  {'type': {'id': 'HP:0001263', 'label': ''}},
  {'type': {'id': 'HP:0002126', 'label': ''}}]}

In [123]:
data['features'][0].split(',')[0]

'HP:0005306'

In [127]:
Dic = {}
for i in range(0, len(data)):
    y = data['id'][i]
    x = data['patient_id'][i]
    z = data['gene_id'][i]
    value = data['features'][i].split(',')
    w = {}
    for j in range(0, len(value)):
        w[j] = {'id': value[j], 'label':{}}
    Dic[i] = {'id': y, 
            'subject': {'id': x}, 
            'phenotypicFeatures': [{'type': w}, 
                                    {'evidence': [{"evidenceCode": {"id": [],
                                                                "label": []},
                                                    "reference": {"id": [],
                                                                "description": []}}]}], 
            'genes': [{'id': z, 'symbol':{}}], 
            'variants': [{'vcfAllele':{"genomeAssembly": [],
                                    "chr": [],
                                    "pos": [],
                                    "ref": [],
                                    "alt": []}},
                         {"zygosity": {"id": [],
                                    "label": []}}],
            "diseases": [{"term": {"id": [],
                                "label": []}}]
            }
Dic

{0: {'id': 'patient_1',
  'subject': {'id': 'Patient:SCV000492960'},
  'phenotypicFeatures': [{'type': {0: {'id': 'HP:0005306', 'label': {}},
     1: {'id': 'HP:0000256', 'label': {}},
     2: {'id': 'HP:0001263', 'label': {}},
     3: {'id': 'HP:0002126', 'label': {}}}},
   {'evidence': [{'evidenceCode': {'id': [], 'label': []},
      'reference': {'id': [], 'description': []}}]}],
  'genes': [{'id': 'Entrez:10000', 'symbol': {}}],
  'variants': [{'vcfAllele': {'genomeAssembly': [],
     'chr': [],
     'pos': [],
     'ref': [],
     'alt': []}},
   {'zygosity': {'id': [], 'label': []}}],
  'diseases': [{'term': {'id': [], 'label': []}}]},
 1: {'id': 'patient_2',
  'subject': {'id': 'Patient:SCV000747622'},
  'phenotypicFeatures': [{'type': {0: {'id': 'HP:0000505', 'label': {}},
     1: {'id': 'HP:0000666', 'label': {}},
     2: {'id': 'HP:0000551', 'label': {}},
     3: {'id': 'HP:0000556', 'label': {}}}},
   {'evidence': [{'evidenceCode': {'id': [], 'label': []},
      'reference':

# Task

In [8]:
feat_list = data['features'][0]
query = feat_list.replace(",", ";")
para = {'q': query}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
concept_id = json_normalize(response['results'])
list_concept_id = concept_id['concept_id'].values.tolist()
list_concept_id = list(map(str, list_concept_id))
str_concept_id = ",".join(list_concept_id).replace(",", ";")
query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
disease = json_normalize(request2['results'])
disease[['concept_code_2', 'concept_name_2', 'ws_ln_ratio']]

,concept_code_2,concept_name_2,ws_ln_ratio
0,MONDO:0000087,polymicrogyria,17.520887
1,MONDO:0022236,colpocephaly,14.990641
2,MONDO:0001149,microcephaly,13.711018
3,MONDO:0017094,cerebral cortical dysplasia,13.336865
4,MONDO:0016231,capillary malformation,12.935155


In [8]:
data['features'][30]

'HP:0000705'

In [5]:
result = {}
for i in range(0, len(data)):
    feat_list = data['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    disease = json_normalize(request2['results'])
    if len(disease) == 0:
        continue
    result[i] = {'subject': data['patient_id'][i], 'top_5_diseases': disease[['concept_code_2', 'concept_name_2', 'ws_ln_ratio']]}

result

JSONDecodeError: [Errno Expecting value] <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.16.1</center>
</body>
</html>
: 0

In [9]:
result = {}
for i in range(0, 10):
    feat_list = data['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    disease = json_normalize(request2['results'])
    if len(disease) == 0:
        continue
    result[i] = {'subject': data['patient_id'][i], 'top_5_diseases': disease[['concept_code_2', 'concept_name_2', 'ws_ln_ratio']]}

result

{0: {'subject': 'Patient:SCV000492960',
  'top_5_diseases':   concept_code_2               concept_name_2  ws_ln_ratio
  0  MONDO:0000087               polymicrogyria    17.520887
  1  MONDO:0022236                 colpocephaly    14.990641
  2  MONDO:0001149                 microcephaly    13.711018
  3  MONDO:0017094  cerebral cortical dysplasia    13.336865
  4  MONDO:0016231       capillary malformation    12.935155},
 1: {'subject': 'Patient:SCV000747622',
  'top_5_diseases':   concept_code_2                              concept_name_2  ws_ln_ratio
  0  MONDO:0007852  palmoplantar keratoderma-deafness syndrome     4.405221
  1  MONDO:0013241              spinocerebellar ataxia type 30     4.390411
  2  MONDO:0007639                        fundus albipunctatus     4.249064
  3  MONDO:0010771                 histiocytoid cardiomyopathy     3.885755
  4  MONDO:0008758     mitochondrial DNA depletion syndrome 4a     3.740244},
 2: {'subject': 'Patient:SCV000599197',
  'top_5_diseases'

In [4]:
disease = []
for i in range(0, 500):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())
    
    
disease

[['polymicrogyria',
  'colpocephaly',
  'microcephaly',
  'cerebral cortical dysplasia',
  'capillary malformation'],
 ['palmoplantar keratoderma-deafness syndrome',
  'spinocerebellar ataxia type 30',
  'fundus albipunctatus',
  'histiocytoid cardiomyopathy',
  'mitochondrial DNA depletion syndrome 4a'],
 ['palmoplantar keratoderma-deafness syndrome',
  'spinocerebellar ataxia type 30',
  'fundus albipunctatus',
  'histiocytoid cardiomyopathy',
  'mitochondrial DNA depletion syndrome 4a'],
 ['occult macular dystrophy',
  'Stargardt disease',
  'inherited retinal dystrophy',
  'severe early-childhood-onset retinal dystrophy',
  'cone-rod dystrophy'],
 ['microcephaly',
  'Prader-Willi syndrome',
  'motor stereotypies',
  'spastic quadriplegia',
  'perinatal asphyxia'],
 ['microcephaly',
  'cerebral cortical dysplasia',
  'polymicrogyria',
  'autosomal recessive primary microcephaly',
  'hereditary spastic paraplegia 17'],
 ['spastic quadriplegia',
  'patent ductus arteriosus',
  'antisy

In [5]:
for i in range(501, 1500):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [7]:
for i in range(1501, 2000):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [10]:
for i in range(2001, 2100):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [11]:
disease2 = []
for i in range(2501, 3400):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease2.append(result['concept_name_2'].values.tolist())

In [12]:
for i in range(3401, 4713):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease2.append(result['concept_name_2'].values.tolist())

In [14]:
for i in range(2101, 2150):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [17]:
for i in range(2151, 2160):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [18]:
for i in range(2161, 2170):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

In [32]:
feat_list = df['features'][2171]
query = feat_list.replace(",", ";")
para = {'q': query}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
concept_id = json_normalize(response['results'])
list_concept_id = concept_id['concept_id'].values.tolist()
list_concept_id = list(map(str, list_concept_id))
str_concept_id = ",".join(list_concept_id).replace(",", ";")
str_concept_id

'80018193;90006731;90100616'

In [27]:
query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
result = json_normalize(request2['results'])

df.loc[2171]

patient_id     Patient:SCV000992208
gene_id                  Entrez:472
features      HP:0006731,HP:0100616
Name: 2171, dtype: object

In [25]:
feat_list = df['features'][2172]
query = feat_list.replace(",", ";")
para = {'q': query}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
concept_id = json_normalize(response['results'])
list_concept_id = concept_id['concept_id'].values.tolist()
list_concept_id = list(map(str, list_concept_id))
str_concept_id = ",".join(list_concept_id).replace(",", ";")
query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
result = json_normalize(request2['results'])

In [22]:
for i in range(2171, 2172):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    para = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
    concept_id = json_normalize(response['results'])
    list_concept_id = concept_id['concept_id'].values.tolist()
    list_concept_id = list(map(str, list_concept_id))
    str_concept_id = ",".join(list_concept_id).replace(",", ";")
    query2 = {'dataset_id': 2, 'concept_id_1': str_concept_id, 'domain_id': 'diseases', 'top_n': 5, 'ascending': 'false'}
    request2 = requests.get('https://rare.cohd.io/api/association/obsExpRatio', params=query2).json()
    result = json_normalize(request2['results'])
    if len(result) == 0:
        continue
    disease.append(result['concept_name_2'].values.tolist())

JSONDecodeError: [Errno Expecting value] <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.16.1</center>
</body>
</html>
: 0